# **Question 8**
Convert an object detection model into a quantized format using any library of your
choice (10 pts)


1.  Run inference with the quantized model (5 pts)
2.  Find the difference in accuracy between the quantized model and the fp32
model (10pts)


# Importing necessary Libraries

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils

In [ ]:
import tensorflow as tf

**Dataset** : Mnist Digit recognisation dataset

**Convolutional Neural Network (CNN) for MNIST Digit Classification**

In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history= model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)
train_loss, train_accuracy = model.evaluate(train_images, train_labels, verbose=2)
print(f"\nTraining Accuracy: {train_accuracy}")

# Print the validation accuracy (if validation data is available)
if 'val_accuracy' in history.history:
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Validation Accuracy: {val_accuracy}")

1688/1688 [==============================] - 20s 12ms/step - loss: 0.2884 - accuracy: 0.9202 - val_loss: 0.1101 - val_accuracy: 0.9712
1875/1875 - 9s - loss: 0.1257 - accuracy: 0.9661 - 9s/epoch - 5ms/step

Training Accuracy: 0.9660500288009644
Validation Accuracy: 0.9711666703224182


1.  A Sequential model is defined with an input layer, reshaping for compatibility with Conv2D layer, a convolutional layer with 12 filters and ReLU activation, a max-pooling layer, and a flattening layer, followed by a dense layer with 10 output units.
2.  The model is compiled using the Adam optimizer and sparse categorical crossentropy loss for digit classification.
Training is performed on the training set with one epoch and a validation split of 10%.
3.  Training accuracy is evaluated and printed after model training.


In [ ]:
 !pip install -q tensorflow
 !pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import tempfile
import os

**Quantization  Training for MNIST Digit Classification**

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_reshape_1 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_1 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_1 (Qua  (None, 13, 13, 12)        1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_1 (QuantizeW  (None, 2028)             


1.   The quantize_model function is applied to the previously defined MNIST digit classification model, creating a quantization-aware version (q_aware_model).
2.   To enable quantization, the quantized model is recompiled with the same configuration as the original model, including the Adam optimizer and sparse categorical crossentropy loss.




In [ ]:
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=50, validation_split=0.1)

Epoch 1/50
2/2 [==============================] - 0s 173ms/step - loss: 0.0717 - accuracy: 0.9822 - val_loss: 0.1851 - val_accuracy: 0.9700
Epoch 2/50
2/2 [==============================] - 0s 139ms/step - loss: 0.0686 - accuracy: 0.9844 - val_loss: 0.1900 - val_accuracy: 0.9700
Epoch 3/50
2/2 [==============================] - 0s 154ms/step - loss: 0.0648 - accuracy: 0.9856 - val_loss: 0.1948 - val_accuracy: 0.9600
Epoch 4/50
2/2 [==============================] - 0s 143ms/step - loss: 0.0623 - accuracy: 0.9867 - val_loss: 0.1868 - val_accuracy: 0.9700
Epoch 5/50
2/2 [==============================] - 0s 133ms/step - loss: 0.0618 - accuracy: 0.9844 - val_loss: 0.1929 - val_accuracy: 0.9600
Epoch 6/50
2/2 [==============================] - 0s 135ms/step - loss: 0.0578 - accuracy: 0.9889 - val_loss: 0.1997 - val_accuracy: 0.9600
Epoch 7/50
2/2 [==============================] - 0s 145ms/step - loss: 0.0553 - accuracy: 0.9900 - val_loss: 0.2198 - val_accuracy: 0.9400
Epoch 8/50
2/2 [====

# **Printing Accuracy**

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9634000062942505
Quant test accuracy: 0.9487000107765198


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.949
Quant TF test accuracy: 0.9487000107765198


In [ ]:
accuracy_difference = train_accuracy - test_accuracy


# Find the difference in accuracy between the quantized model and the fp32 model

In [ ]:
print(f"Original Model Accuracy: {train_accuracy}")
print(f"Quantized Model Accuracy: { test_accuracy}")
print(f"Accuracy Difference: {accuracy_difference}")

Original Model Accuracy: 0.9660500288009644
Quantized Model Accuracy: 0.949
Accuracy Difference: 0.0170500288009644
